#### Importing libraries

In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import geocoder
import folium
from geopy.geocoders import Nominatim

import random 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import matplotlib.cm as cm
import matplotlib.colors as colors
import json 
from pandas.io.json import json_normalize 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


#### Scraping data from the website

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).content
soup = BeautifulSoup(html_data,"html5lib")
t = soup.title

In [3]:
tables = soup.find("table")

#### Creating a data frame with the required information

In [4]:
table_contents = []

In [5]:
for row in tables.findAll('td'):
    cell = {}
    if 'Not assigned' in row.span.text:
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [6]:
df = pd.DataFrame(table_contents)
df.sort_values(by = 'PostalCode', inplace = True)

In [7]:
geo_data = pd.read_csv('/Users/test/Downloads/Geospatial_Coordinates.csv')

In [8]:
df['Latitude'] = geo_data['Latitude']
df['Longitude'] = geo_data['Longitude']

In [9]:
df.sort_index(inplace = True)

#### Use geopy library to get the latitude and longitude values of Toronto.


In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
tor_lat = location.latitude
tor_long = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_lat, tor_long))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [11]:
map_toronto = folium.Map(location=[tor_lat, tor_long], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)

#### Map of boroughs that contain the word Toronto 

In [13]:
map_borough = folium.Map(location=[tor_lat, tor_long], zoom_start=12)

# add markers to map
for lat, lng, bor, label in zip(toronto_data['Latitude'], toronto_data['Longitude'],toronto_data['Borough'],toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough)  
    
map_borough

#### Define Foursquare Credentials and Version


In [14]:
CLIENT_ID = '1G0J0TCQ0VR4A3CPKSYGTQFZWAL3WNYLKCYFQCJB2MQWUHBI' # your Foursquare ID
CLIENT_SECRET = 'JEHA4U1Z1BX3BZIPAYRWRPSV5QMC3S0BHOIEVTI1RZMZFXJZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Let's explore the fifteenth neighborhood in our dataframe.


In [15]:
tor_grouped = toronto_data.groupby('Neighborhood').mean().reset_index()

In [16]:
toronto_data.loc[14, 'Neighborhood']

'Brockton, Parkdale Village, Exhibition Place'

In [17]:
lat = toronto_data.loc[14, 'Latitude'] # neighborhood latitude value
long = toronto_data.loc[14, 'Longitude'] # neighborhood longitude value
name = toronto_data.loc[14, 'Neighborhood'] # neighborhood name

In [18]:
radius = 800

#### Now, let's get the top 100 venues that are in Brockton, Parkdale Village, Exhibition Place within a radius of 800 meters.


In [19]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)

In [20]:
results = requests.get(url).json()

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-22-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Queen Books,Bookstore,43.660651,-79.342267
2,Te Aro,Coffee Shop,43.661373,-79.338577
3,Mercury Espresso Bar,Coffee Shop,43.660806,-79.341241
4,Reliable Halibut and Chips,Seafood Restaurant,43.660874,-79.340938


## 2. Explore Neighborhoods in Toronto


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.


In [24]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [25]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
...,...,...,...,...,...,...,...
743,Church and Wellesley,43.696319,-79.532242,Dollarama,43.691945,-79.531593,Discount Store
744,Enclave of M4L,43.688905,-79.554724,Subway,43.692936,-79.557351,Sandwich Place
745,Enclave of M4L,43.688905,-79.554724,Wincott Park,43.689301,-79.551802,Park
746,Enclave of M4L,43.688905,-79.554724,TTC Bus 45 Kipling,43.690907,-79.557066,Bus Line


In [26]:
len(toronto_venues['Venue Category'].unique())

197

## 3. Analyze Each Neighborhood


In [27]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"] 

i=toronto_onehot.columns.get_loc('Neighborhood') #getting the index of "Neighborhood"
fixed_columns = [toronto_onehot.columns[i]] + list(toronto_onehot.columns[:i]) + list(toronto_onehot.columns[i+1:])
toronto_onehot = toronto_onehot[fixed_columns]

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Let's print each neighborhood along with the top 5 most common venues


In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.08
1              Brewery  0.06
2                 Café  0.06
3  American Restaurant  0.06
4            Gastropub  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Light Rail Station  0.13
1                  Park  0.07
2         Garden Center  0.07
3  Fast Food Restaurant  0.07
4         Burrito Place  0.07


----Central Bay Street----
            venue  freq
0        Pharmacy   0.2
1     Coffee Shop   0.2
2     Supermarket   0.2
3  Discount Store   0.2
4     Pizza Place   0.2


----Christie----
                 venue  freq
0                 Park   0.5
1    Food & Drink Shop   0.5
2  Monument / Landmark   0.0
3         Liquor Store   0.0
4               Lounge   0.0


----Church and Wellesley----
                       venue  freq
0                Pizza Place  0.14
1        

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Café,Yoga Studio,Clothing Store,Seafood Restaurant,Italian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Pizza Place,Brewery,Skate Park,Restaurant,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Auto Workshop
2,Central Bay Street,Coffee Shop,Pharmacy,Pizza Place,Supermarket,Discount Store,Yoga Studio,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,Christie,Park,Food & Drink Shop,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,Church and Wellesley,Coffee Shop,Pizza Place,Intersection,Sandwich Place,Chinese Restaurant,Discount Store,Middle Eastern Restaurant,Yoga Studio,Diner,Dumpling Restaurant


## 4. Cluster Neighborhoods


In [32]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 3, 0, 3, 0, 0, 4, 0], dtype=int32)

In [33]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
merged_torontodata = toronto_data

merged_torontodata = merged_torontodata.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how ='inner')

merged_torontodata.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Mexican Restaurant,Breakfast Spot,Donut Shop,Intersection,Medical Center,Bank,Restaurant,Rental Car Location,Electronics Store,Yoga Studio
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,4,General Entertainment,Skating Rink,Café,College Stadium,Convenience Store,Cosmetics Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Fast Food Restaurant,Noodle House,Breakfast Spot,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Chinese Restaurant,Bank,Yoga Studio
3,M4E,East Toronto,The Beaches,43.786947,-79.385975,4,Café,Japanese Restaurant,Chinese Restaurant,Bank,Discount Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259,0,Coffee Shop,Pharmacy,Pizza Place,Supermarket,Discount Store,Yoga Studio,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [34]:
merged_torontodata.dropna(inplace = True)

In [35]:
merged_torontodata.isnull().sum()

PostalCode                0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [36]:
map_clusters = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_torontodata['Latitude'], merged_torontodata['Longitude'], merged_torontodata['Neighborhood'], merged_torontodata['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


#### Cluster 1


In [37]:
merged_torontodata.loc[merged_torontodata['Cluster Labels'] == 0, merged_torontodata.columns[[1] + list(range(5, merged_torontodata.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Mexican Restaurant,Breakfast Spot,Donut Shop,Intersection,Medical Center,Bank,Restaurant,Rental Car Location,Electronics Store,Yoga Studio
2,Downtown Toronto,0,Fast Food Restaurant,Noodle House,Breakfast Spot,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Chinese Restaurant,Bank,Yoga Studio
5,Downtown Toronto,0,Coffee Shop,Pharmacy,Pizza Place,Supermarket,Discount Store,Yoga Studio,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
7,Downtown Toronto,0,Airport,Electronics Store,Park,Airport Gate,Diner,Ethiopian Restaurant,Airport Food Court,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,East YorkEast Toronto,0,Pizza Place,Gastropub,Gym / Fitness Center,Intersection,Flea Market,Bank,Athletics & Sports,Breakfast Spot,Pharmacy,Pet Store
10,Downtown Toronto,0,Park,Curling Ice,Beer Store,Intersection,Skating Rink,Coworking Space,Dog Run,Construction & Landscaping,Ethiopian Restaurant,Electronics Store
11,West Toronto,0,Pub,Coffee Shop,Trail,Health Food Store,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
12,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Pub
13,Downtown Toronto,0,Sandwich Place,Fast Food Restaurant,Pet Store,Pub,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Liquor Store,Pizza Place,Gym
14,West Toronto,0,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Café,Yoga Studio,Clothing Store,Seafood Restaurant,Italian Restaurant


#### Cluster 2


In [38]:
merged_torontodata.loc[merged_torontodata['Cluster Labels'] == 1, merged_torontodata.columns[[1] + list(range(5, merged_torontodata.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,1,Park,Baseball Field,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
36,Downtown Toronto,1,Baseball Field,Food Service,Yoga Studio,Discount Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 3


In [39]:
merged_torontodata.loc[merged_torontodata['Cluster Labels'] == 2, merged_torontodata.columns[[1] + list(range(5, merged_torontodata.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Downtown Toronto,2,Pizza Place,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 4


In [40]:
merged_torontodata.loc[merged_torontodata['Cluster Labels'] == 3, merged_torontodata.columns[[1] + list(range(5, merged_torontodata.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,3,Park,Food & Drink Shop,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
16,Downtown Toronto,3,Park,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
24,Central Toronto,3,Park,Pool,Women's Store,Airport Gate,Airport Lounge,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 5


In [41]:
merged_torontodata.loc[merged_torontodata['Cluster Labels'] == 4, merged_torontodata.columns[[1] + list(range(5, merged_torontodata.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,General Entertainment,Skating Rink,Café,College Stadium,Convenience Store,Cosmetics Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,East Toronto,4,Café,Japanese Restaurant,Chinese Restaurant,Bank,Discount Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
8,West Toronto,4,Grocery Store,Hotel,Park,Shopping Mall,Bank,Yoga Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
25,West Toronto,4,Grocery Store,Café,Park,Baby Store,Candy Store,Nightclub,Coffee Shop,Italian Restaurant,Restaurant,Dumpling Restaurant
28,West Toronto,4,Convenience Store,Grocery Store,Breakfast Spot,Yoga Studio,Discount Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
